In [10]:
from dotenv import load_dotenv, find_dotenv

print(load_dotenv(find_dotenv()))


import openai

functions = [
{
    "name": "get_weather",
    "description": "Get the weather forecast for a  specific city",
    "parameters" : {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "Name of the city"
            },
            "unit": {
                "type": "string",
                "enum": ["Celsius", "Farenheit"],
                "description": "Temperature unit"         
            }
        },
        "required": ["city", "unit"]

    }
}
]

response = openai.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "user", "content": "What is the weather in New York in Celsius?"}
    ],
    functions= functions,
    function_call={"name": "get_weather"}
)

arguments_str = response.choices[0].message.function_call.arguments
print(arguments_str)

True
{"city":"New York","unit":"Celsius"}


In [21]:
functions = [
    {
        "name": "send_email",
        "description": "Send an email with a subject and body",
        "parameters": {
            "type": "object",
            "properties": {
                "to": {"type": "string", "description": "Recipient email address"},
                "subject": {"type": "string", "description": "Email subject"},
                "body": {"type": "string", "description": "Email body text"}
            },
            "required": ["to", "subject", "body"]
        }
    }
]

response = openai.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "user", "content": "Send an email to elijahrobot@gmail.com with the subject 'Showing Demo to ELijah' and say 'HOw do you like this demo.'"}
    ],
    functions=functions,
    function_call={"name": "send_email"}
)

arguments_str = response.choices[0].message.function_call.arguments
print(arguments_str)

import smtplib
from email.mime.text import MIMEText

# Function to send email
def send_email(recipient, subject, body):
    # Configure your email server (this example uses a Gmail SMTP server)
    smtp_server = "smtp.gmail.com"
    smtp_port = 587
    sender_email = "batth.maninder@gmail.com"
    sender_password = "hsscsxbzioyhzcpp"

    # Create the email message
    msg = MIMEText(body)
    msg["Subject"] = subject
    msg["From"] = sender_email
    msg["To"] = recipient

    # Send the email
    with smtplib.SMTP(smtp_server, smtp_port) as server:
        server.starttls()
        server.login(sender_email, sender_password)
        server.sendmail(sender_email, recipient, msg.as_string())
        print("Email sent successfully!")


# Call the function with data from LLM response
import json
email_data = json.loads(arguments_str)

send_email(email_data["to"], email_data["subject"], email_data["body"])


{"to":"elijahrobot@gmail.com","subject":"Showing Demo to ELijah","body":"HOw do you like this demo."}
Email sent successfully!


In [22]:
import openai
from dotenv import load_dotenv, find_dotenv

# Load environment variables
load_dotenv(find_dotenv())

# Define functions: one for weather information, and one for currency exchange rates
functions = [
    {
        "name": "get_weather",
        "description": "Get the weather forecast for a specific city",
        "parameters": {
            "type": "object",
            "properties": {
                "city": {
                    "type": "string",
                    "description": "Name of the city"
                },
                "unit": {
                    "type": "string",
                    "enum": ["Celsius", "Fahrenheit"],
                    "description": "Temperature unit"
                }
            },
            "required": ["city", "unit"]
        }
    },
    {
        "name": "get_exchange_rate",
        "description": "Get the exchange rate between two currencies",
        "parameters": {
            "type": "object",
            "properties": {
                "from_currency": {
                    "type": "string",
                    "description": "The currency to convert from, e.g., 'USD'"
                },
                "to_currency": {
                    "type": "string",
                    "description": "The currency to convert to, e.g., 'EUR'"
                }
            },
            "required": ["from_currency", "to_currency"]
        }
    }
]

# Prompts: one asks for weather, the other for currency exchange
prompts = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "What is the weather in Paris in Celsius?"},
    {"role": "user", "content": "What is the exchange rate from USD to EUR?"}
]

# Run the conversation for each prompt to see which function the LLM selects
for prompt in prompts[1:]:
    # Include the system message for context
    messages = [prompts[0], prompt]
    
    # Make the request with function_call set to "auto"
    response = openai.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        functions=functions,
        function_call="auto"  # Allow the model to decide which function to call
    )

    # Output the function call decision
    if response.choices[0].message. function_call:
        function_name = response.choices[0].message.function_call.name
        function_args = response.choices[0].message.function_call.arguments
        
        print(f"Prompt: {prompt['content']}")
        print(f"Selected Function: {function_name}")
        print(f"Arguments: {function_args}")
        print("\n" + "="*50 + "\n")
    else:
        # If no function is called, output the model's response
        print(f"Prompt: {prompt['content']}")
        print("Response:", response.choices[0].message.content)
        print("\n" + "="*50 + "\n")


Prompt: What is the weather in Paris in Celsius?
Selected Function: get_weather
Arguments: {"city":"Paris","unit":"Celsius"}


Prompt: What is the exchange rate from USD to EUR?
Selected Function: get_exchange_rate
Arguments: {"from_currency":"USD","to_currency":"EUR"}




In [20]:
import requests

def google_search(query, api_key, cse_id, num_results=5):
    # Define the endpoint and parameters for the API request
    url = "https://www.googleapis.com/customsearch/v1"
    params = {
        "q": query,                # The search query
        "key": api_key,            # Your API key
        "cx": "97e237aec9f16471a",              # Your custom search engine ID
        "num": num_results         # Number of search results to return
    }
    
    # Send the GET request to the Google Custom Search API
    response = requests.get(url, params=params)
    
    # Raise an error if the request was unsuccessful
    response.raise_for_status()
    
    # Parse the response JSON and return the search results
    search_results = response.json().get("items", [])
    return search_results

# Set up your API key and Custom Search Engine ID
api_key = ""
cse_id = ""

# Example usage of the function
query = "Python programming tutorials"
try:
    results = google_search(query, api_key, cse_id)
    for i, result in enumerate(results, start=1):
        title = result.get("title")
        link = result.get("link")
        snippet = result.get("snippet")
        print(f"{i}. {title}\n   {link}\n   {snippet}\n")
except requests.exceptions.HTTPError as err:
    print(f"HTTP error occurred: {err}")
except Exception as err:
    print(f"Other error occurred: {err}")


1. The Python Tutorial — Python 3.13.0 documentation
   https://docs.python.org/3/tutorial/index.html
   Python is an easy to learn, powerful programming language. It has efficient high-level data structures and a simple but effective approach to ...

2. Python Tutorial
   https://www.w3schools.com/python/
   Learn Python. Python is a popular programming language. Python can be used on a server to create web applications. Start learning Python now.

3. Python For Beginners | Python.org
   https://www.python.org/about/gettingstarted/
   Fortunately an experienced programmer in any programming ... There is a list of tutorials suitable for experienced programmers on the BeginnersGuide/Tutorials ...

4. Best Python tutorial for beginners in 2024? : r/learnpython
   https://www.reddit.com/r/learnpython/comments/1ajlvog/best_python_tutorial_for_beginners_in_2024/
   Feb 5, 2024 ... I have been using codecademy and study.com for the basics. then I have chat gpt teach me by having it list 30 b